# Choice simulation work

Sam Maurer, October 2018

This notebook contains benchmarks, feature development, and testing for ChoiceModels PR #TK, related to issue #TK.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import choicemodels
print(choicemodels.__version__)

0.2.dev3


### Benchmark df.apply vs matrix math for chooser-level random draws

There's no `numpy` function to perform simultaneous random draws from K distinct probability distributions, which we often need to do to simulate choices for K choosers.

Fletcher wrote an implementation using matrix math for `urbansim.urbanchoice.mnl`, which I refactored and generalized in `choicemodels.tools`. 

But I realized that in other places, like `urbansim.models.dcm`, we use `df.apply` for similar operations. This seems cleaner and more easily maintainable, and i'm curious how the performance compares. Maybe the matrix math implementation is only needed for things like GPU acceleration?

In [3]:
from choicemodels.tools import simulate_choices

In [4]:
def generate_probs(n_obs, n_alts):
    n_obs = int(n_obs)
    n_alts = int(n_alts)
    
    d = {'oid': np.repeat(np.arange(n_obs), n_alts),
         'aid': np.tile(np.arange(n_alts), n_obs),
         'probs': np.random.random(n_obs * n_alts)}

    df = pd.DataFrame(d)
    df['probs'] = df.probs.div(df.groupby('oid').probs.transform('sum'))
    return df.set_index(['oid','aid']).probs

print(generate_probs(2,3))

oid  aid
0    0      0.337944
     1      0.494199
     2      0.167857
1    0      0.151682
     1      0.238844
     2      0.609474
Name: probs, dtype: float64


In [5]:
probs = generate_probs(1e4, 100)

#### 1. Matrix implementation from choicemodels

In [6]:
%%time
c = simulate_choices(probs)
print(len(c))

10000
CPU times: user 21.3 ms, sys: 4.47 ms, total: 25.8 ms
Wall time: 24.2 ms


#### 2. df.apply

In [7]:
df = pd.DataFrame(probs).reset_index()

In [8]:
%%time
c = df.groupby('oid').apply(lambda x: np.random.choice(x.aid, p=x.probs))
print(len(c))

10000
CPU times: user 1.11 s, sys: 10.7 ms, total: 1.12 s
Wall time: 1.11 s


#### 3. Try keeping indexes to make it faster

In [9]:
def mkchoice(probs):
    return np.random.choice(probs.index.values, p=probs)

In [10]:
%%time
c = probs.groupby(level='oid', sort=False).apply(mkchoice)
print(len(c))

10000
CPU times: user 3.4 s, sys: 26.1 ms, total: 3.42 s
Wall time: 3.41 s


**df.apply is way slower!! At least 50x. We should use the matrix math implementation everywhere**

#### 4. What about np.random.choice with single probability distribution?

In [11]:
probs = np.random.random(100)
probs = probs/probs.sum()

In [13]:
%%time
c = np.random.choice(np.arange(100), size=int(1e4), replace=True, p=probs)

CPU times: user 1.47 ms, sys: 1.49 ms, total: 2.96 ms
Wall time: 1.73 ms


That's the way to go when there's a single probability distribution